<a href="https://colab.research.google.com/github/deanhoperobertson/Masters-/blob/master/Thesis/Code/Bi_LSTM_CRF_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bi-LSTM-CRF Model

In [18]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro (1).py


In [19]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-12apv_we
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-12apv_we
  Stored in directory: /tmp/pip-ephem-wheel-cache-jymoz0xg/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [20]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [25]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words=list(set(words))
n_words = len(words)
print("Number of words in the dataset: ", n_words)
tags = list(set(tags))
n_tags = len(tags)
print("Number of Labels: ", n_tags)

Number of words in the dataset:  27316
Number of Labels:  5


In [26]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 19957


In [29]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["LOC"]))

The labels LOC (location) is identified by the index: 2


In [30]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [34]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X

array([[ 6100, 19957, 22644, ...,     0,     0,     0],
       [ 2896, 13614,     0, ...,     0,     0,     0],
       [13089,  8553,     0, ...,     0,     0,     0],
       ...,
       [13405, 14701, 23645, ...,     0,     0,     0],
       [ 7645, 10133,     0, ...,     0,     0,     0],
       [14534, 13687,  1328, ...,     0,     0,     0]], dtype=int32)

In [59]:
# Convert Tag/Label to tag_index
y = [[tag2idx[(w[1])] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y

ValueError: ignored

In [113]:
labels=[]
for sent in train:
  labels.append([w[1] for w in sent])
  
test=[partial_tags(w) for w in labels]

y=[[tag2idx[w] for w in s] for s in test]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y

array([[5, 1, 4, ..., 0, 0, 0],
       [3, 3, 0, ..., 0, 0, 0],
       [2, 1, 0, ..., 0, 0, 0],
       ...,
       [5, 1, 5, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [5, 1, 5, ..., 0, 0, 0]], dtype=int32)

In [0]:

# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [115]:
#split into test and train subsets
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
X_tr.shape,np.array(y_tr).shape, X_te.shape, np.array(y_te).shape

((12636, 124), (12636, 124, 6), (1405, 124), (1405, 124, 6))

## Building LSTM-CRF Model

In [116]:
# Model definition
EMBEDDING=40

input = Input(shape=(MAX_LEN,))
model = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)

model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)

model = TimeDistributed(Dense(10, activation="relu"))(model)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model)  # output
model = Model(input, out)

W0722 21:18:42.555502 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 21:18:42.601827 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 21:18:42.611215 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 21:18:42.842855 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 21:18:42.859333 

In [117]:
model.compile(optimizer="rmsprop", loss=crf_loss)#, metrics=["accuracy"])
model.summary()

W0722 21:19:02.414615 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 124, 40)           1092720   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 124, 100)          36400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 124, 10)           1010      
_________________________________________________________________
crf_1 (CRF)                  (None, 124, 6)            114       
Total params: 1,130,244
Trainable params: 1,130,244
Non-trainable params: 0
_________________________________________________________________


In [118]:
%%time
BATCH_SIZE = 500
EPOCHS=10

history = model.fit(X_tr, np.array(y_tr), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=1)

W0722 21:19:36.923455 140044418230144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 11372 samples, validate on 1264 samples
Epoch 1/10
11372/11372 [==============================] - 26s 2ms/step - loss: 32.9888 - val_loss: 31.7149
Epoch 2/10
11372/11372 [==============================] - 20s 2ms/step - loss: 32.6161 - val_loss: 31.5318
Epoch 3/10
11372/11372 [==============================] - 20s 2ms/step - loss: 32.4538 - val_loss: 31.3920
Epoch 4/10
11372/11372 [==============================] - 21s 2ms/step - loss: 32.3026 - val_loss: 31.2575
Epoch 5/10
11372/11372 [==============================] - 20s 2ms/step - loss: 32.1732 - val_loss: 31.1705
Epoch 6/10
11372/11372 [==============================] - 20s 2ms/step - loss: 32.0908 - val_loss: 31.1237
Epoch 7/10
11372/11372 [==============================] - 20s 2ms/step - loss: 32.0315 - val_loss: 31.0810
Epoch 8/10
11372/11372 [==============================] - 20s 2ms/step - loss: 31.9827 - val_loss: 31.0516
Epoch 9/10
11372/11372 [==============================] - 20s 2ms/step - loss: 31.9400 - val_lo

## Predict on Training Set

In [130]:
pred = model.predict(X_tr, verbose=1)

12636/12636 [==============================] - 147s 12ms/step


In [131]:
# TRain Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_tr, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.76      0.87      0.81      7492
        MISC       0.92      0.70      0.79      4170
           O       0.99      1.00      0.99    152857
         ORG       0.86      0.80      0.83      9048
         PAD       1.00      1.00      1.00   1383300
         PER       0.88      0.86      0.87      9997

    accuracy                           1.00   1566864
   macro avg       0.90      0.87      0.88   1566864
weighted avg       1.00      1.00      1.00   1566864

0.9962031481572303


In [132]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

         LOC       0.76      0.87      0.81      7492
         PER       0.88      0.86      0.87      9997
        MISC       0.92      0.70      0.79      4170
         ORG       0.86      0.80      0.83      9048

   micro avg       0.84      0.82      0.83     30707
   macro avg       0.85      0.81      0.83     30707
weighted avg       0.85      0.82      0.83     30707

0.8332283387061152


## Predict on Test Set